In [1]:
import glob
import json
import re
import pandas as pd
import os

In [2]:
# Define a function to get column names from schemas.json
def get_column_names(schemas: dict, ds_name: str, sorting_key='column_position'):
    column_details = schemas[ds_name]
    columns = sorted(column_details, key=lambda col: col[sorting_key], reverse=False)
    return [col['column_name'] for col in columns]

In [3]:
# read csv file by using the column names in schema
def read_csv(file, schemas):
    ds_name = re.split('[/]', file)[-2]
    columns = get_column_names(schemas, ds_name)
    df = pd.read_csv(file, names=columns)
    return df

In [4]:
# convert csv file to json file using pandas dataframe
def to_json(df, trg_base_dir, ds_name, file_name):
    json_file_path = f'{trg_base_dir}/{ds_name}/{file_name}'
    os.makedirs(f'{trg_base_dir}/{ds_name}', exist_ok=True)
    df.to_json(
        json_file_path,
        orient='records',
        lines=True
    )

In [21]:
# file converter function
def file_converter(src_base_dir, trg_base_dir, ds_name):
    schemas = json.load(open(f'{src_base_dir}/schemas.json'))
    files = glob.glob(f'{src_base_dir}/{ds_name}/part-*')

    for file in files:
        df = read_csv(file, schemas)
        file_name = re.split('[/]', file)[-1]
        to_json(df, trg_base_dir, ds_name, file_name)


In [22]:
def process_files(ds_names=None):
    src_base_dir = '../data/retail_db'
    trg_base_dir = '../data/retail_db_json'
    schemas = json.load(open(f'{src_base_dir}/schemas.json'))
    if not ds_names:
        ds_names = schemas.keys()
    for ds_name in ds_names:
        print(f'Processing {ds_name}')
        file_converter(src_base_dir, trg_base_dir, ds_name)

In [23]:
process_files()

Processing departments
Processing categories
Processing orders
Processing products
Processing customers
Processing order_items


In [18]:
glob.glob(f'../data/retail_db/orders/part-*')

['../data/retail_db/orders/part-00000']

In [9]:
#process_files(['departments','categories'])

Processing departments
Processing categories
